In [1]:
# Include required libs
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import metrics
import random
from sklearn import preprocessing
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
def load_train_and_test_data():
    train_features=pd.read_csv('TrainingSetValues.csv',parse_dates=True)
    train_labels=pd.read_csv('TrainingSetLabels.csv')
    to_predict_features=pd.read_csv('TestSetValues.csv',parse_dates=True)
    
    # merge training features and labels
    #training_data = pd.merge(train_features, train_labels, how='inner', on=['id'])
    
    return train_features, train_labels, to_predict_features

In [3]:
# impute median values for 0 amount_tsh
def impute_missing_amount_tsh(df):
    df.amount_tsh[df.amount_tsh <= 0] = np.median(df.amount_tsh[df.amount_tsh > 0])
    return df

In [4]:
# impute median values for 0 gps height
def impute_missing_gps_height(df):
    df.gps_height[df.gps_height <= 0] = np.median(df.gps_height[df.gps_height > 0])
    return df

In [5]:
# impute median values for 0 population
def impute_missing_population(df):
    df.population[df.population <= 0] = np.median(df.population[df.population > 0])
    return df

In [6]:
# transform population into categories
def label_population(row):
    if row['population'] <=0:
        return 'NA'
    elif row['population'] >= 1 and row['population'] <= 40:
        return 'A'
    elif row['population'] >= 41 and row['population'] <= 67:
        return 'B'
    elif row['population'] >= 68 and row['population'] <= 99:
        return 'C'
    elif row['population'] >= 100 and row['population'] <= 131:
        return 'D'
    elif row['population'] >= 132 and row['population'] <= 175:
        return 'E'
    elif row['population'] >= 176 and row['population'] <= 219:
        return 'F'
    elif row['population'] >= 220 and row['population'] <= 259:
        return 'G'
    elif row['population'] >= 260 and row['population'] <= 349:
        return 'H'
    elif row['population'] >= 350 and row['population'] <= 448:
        return 'I'
    elif row['population'] >= 449 and row['population'] <= 598:
        return 'J'
    elif row['population'] >= 599 and row['population'] <= 1290:
        return 'K'
    elif row['population'] >= 1291:
        return 'L'

def transform_population_into_categories(df):
    df['population_cat'] = df.apply(label_population, axis=1)
    return df

In [7]:
# impute median values for construction year
def impute_missing_construction_year(df):
    df.construction_year[df.construction_year <= 0] = np.median(df.construction_year[df.construction_year > 0])
    return df

In [8]:
# impute missing booleans with false and convert each value to float or integer
def impute_missing_booleans(df, colname):
    df[colname].fillna(False, inplace = True)
    df[colname] = df[colname].apply(lambda x: float(x))
    return df

In [9]:
def dates(X_train, X_test):
    """
    date_recorded: this might be a useful variable for this analysis, although the year itself would be useless 
    in a practical scenario moving into the future. 
    We will convert this column into a datetime, and we will also 
    create 'year_recorded' and 'month_recorded' columns just in case those levels prove to be useful. 
    A visual inspection of both casts significant doubt on that possibility, but we'll proceed for now. 
    We will delete date_recorded itself, since random forest cannot accept datetime
    """
    for i in [X_train, X_test]:
        i['date_recorded'] = pd.to_datetime(i['date_recorded'])
        i['year_recorded'] = i['date_recorded'].apply(lambda x: x.year)
        i['month_recorded'] = i['date_recorded'].apply(lambda x: x.month)
        i['date_recorded'] = (pd.to_datetime(i['date_recorded'])).apply(lambda x: x.toordinal())
    return X_train, X_test


In [10]:
# Since random forest doesnt work on datetime we will cconvert into month and year.
# But, also if we simply convert month into numerical values, it doesnt work well because 
# there may be big distance between Jan and December and also between 1970 to  2010, to take an example
# So its better to one hot encode them after transforming the date to month and year
def transform_date_recorded_to_month_and_year(df):
    df['date_recorded'] = pd.to_datetime(df['date_recorded'])
    df['year_recorded'] = df['date_recorded'].apply(lambda x: x.year)
    df['month_recorded'] = df['date_recorded'].apply(lambda x: x.month)
    df['date_recorded'] = (pd.to_datetime(df['date_recorded'])).apply(lambda x: x.toordinal())
    return df

In [11]:
def dates2(X_train, X_test):
    """
    Turn year_recorded and month_recorded into dummy variables
    """
    for z in ['month_recorded', 'year_recorded']:
        X_train[z] = X_train[z].apply(lambda x: str(x))
        X_test[z] = X_test[z].apply(lambda x: str(x))
        good_cols = [z+'_'+i for i in X_train[z].unique() if i in X_test[z].unique()]
        X_train = pd.concat((X_train, pd.get_dummies(X_train[z], prefix = z)[good_cols]), axis = 1)
        X_test = pd.concat((X_test, pd.get_dummies(X_test[z], prefix = z)[good_cols]), axis = 1)
        del X_test[z]
        del X_train[z]
    return X_train, X_test

In [12]:
# One Hot encode year and month recorded (first convert the month and year to string beafore OHEing)
# Also delete the original ones
def ohe_month_and_year_recorded(df_train, df_test):
    for col in ['month_recorded', 'year_recorded']:
        df_train[col] = df_train[col].apply(lambda x: str(x))
        df_test[col] = df_test[col].apply(lambda x: str(x))
        ohe_cols_postfix = [col + '_' + i for i in df_train[col].unique() if i in df_test[col].unique()]
        df_train = pd.concat((df_train, pd.get_dummies(df_train[col], prefix = col)[ohe_cols_postfix]), axis = 1)
        df_test = pd.concat((df_test, pd.get_dummies(df_test[col], prefix = col)[ohe_cols_postfix]), axis = 1)
        del df_test[col]
        del df_train[col]
    return df_train, df_test

In [13]:
def locs(X_train, X_test):
    """
    fill in the nulls for ['longitude', 'latitude', 'gps_height', 'population'] by using means from 
    ['subvillage', 'district_code', 'basin'], and lastly the overall mean
    """
    trans = ['longitude', 'latitude', 'gps_height', 'population']
    for i in [X_train, X_test]:
        i.loc[i.longitude == 0, 'latitude'] = 0
    for z in trans:
        for i in [X_train, X_test]:
            i[z].replace(0., np.NaN, inplace = True)
            i[z].replace(1., np.NaN, inplace = True)
        
        for j in ['subvillage', 'district_code', 'basin']:
        
            X_train['mean'] = X_train.groupby([j])[z].transform('mean')
            X_train[z] = X_train[z].fillna(X_train['mean'])
            o = X_train.groupby([j])[z].mean()
            fill = pd.merge(X_test, pd.DataFrame(o), left_on=[j], right_index=True, how='left').iloc[:,-1]
            X_test[z] = X_test[z].fillna(fill)
        
        X_train[z] = X_train[z].fillna(X_train[z].mean())
        X_test[z] = X_test[z].fillna(X_train[z].mean())
        del X_train['mean']
    return X_train, X_test

In [14]:
# function to drop columns from a dataframe
def drop_columns(df, cols_to_drop):
    for col in cols_to_drop:
        del df[col]
    return df

In [15]:
def gini(p):
    return 1-(p**2 + (1-p)**2)

In [16]:
def small_n2(X_train, X_test):
    cols = [i for i in X_train.columns if type(X_train[i].iloc[0]) == str]
    X_train[cols] = X_train[cols].where(X_train[cols].apply(lambda x: x.map(x.value_counts())) > 100, "other")
    for column in cols:
        for i in X_test[column].unique():
            if i not in X_train[column].unique():
                X_test[column].replace(i, 'other', inplace=True)
    return X_train, X_test

In [17]:
def lda(X_train, X_test, y_train, cols=['population', 'gps_height', 'latitude', 'longitude']):
    sc = StandardScaler()
    X_train_std = sc.fit_transform(X_train[cols])
    X_test_std = sc.transform(X_test[cols])
    lda = LDA(n_components=None)
    X_train_lda = lda.fit_transform(X_train_std, y_train.values.ravel())
    X_test_lda = lda.transform(X_test_std)
    X_train = pd.concat((pd.DataFrame(X_train_lda), X_train), axis=1)
    X_test = pd.concat((pd.DataFrame(X_test_lda), X_test), axis=1)
    for i in cols:
        del X_train[i]
        del X_test[i]
    return X_train, X_test

In [18]:
def one_hot_encode(df_train, df_test):
    columns = [i for i in df_train.columns if type(df_train[i].iloc[0]) == str]
    for column in columns:
        df_train[column].fillna('NULL', inplace = True)
        ohe_cols = [column+'_'+i for i in df_train[column].unique() if i in df_test[column].unique()]
        df_train = pd.concat((df_train, pd.get_dummies(df_train[column], prefix = column)[ohe_cols]), axis = 1)
        df_test = pd.concat((df_test, pd.get_dummies(df_test[column], prefix = column)[ohe_cols]), axis = 1)
        del df_train[column]
        del df_test[column]
    return df_train, df_test

In [19]:
def one_hot_encode1(df):
    columns = [x for x in df.columns if type(df[x].iloc[0]) == str]
    for column in columns:
        df[column].fillna('NULL', inplace = True)
        ohe_cols = [column + '_' + x for x in df[column].unique() if x in df[column].unique()]
        df = pd.concat((df, pd.get_dummies(df[column], prefix = column)[ohe_cols]), axis = 1)
        del df[column]
    return df

In [20]:
# Helper function for column values analysis
def analyze_unique_values_for_column(df, colname):
    unique_col_vals = df[colname].unique()
    tmp_str = "Unique " + colname + "s:"
    print("****************************")
    print(tmp_str, unique_col_vals.size)
    print("****************************")

In [21]:
# Helper function for column values analysis
def analyze_in_detail_unique_values_for_column(df, colname):
    unique_col_vals = df[colname].unique()
    tmp_str = "Unique " + colname + "s:"
    print("****************************")
    print(tmp_str, unique_col_vals.size)
    print("****************************")
    lessthan10 = 0
    lessthan20 = 0
    lessthan30 = 0
    lessthan50 = 0
    lessthan100 = 0
    for val in unique_col_vals:
        cnt = df[df[colname] == val][colname].count()
        print(val, cnt) # uncomment this line if you want to see the count of each colname-value
        if(cnt < 10):
            lessthan10 +=1     
            print(val, cnt)
        elif(cnt < 20):
            lessthan20 +=1
        elif(cnt < 30):
            lessthan30 +=1
        elif(cnt < 50):
            lessthan50 +=1

    print("lessthan50: ", lessthan50 )
    print("lessthan30: ", lessthan30 )
    print("lessthan20: ", lessthan20 )
    print("lessthan10: ", lessthan10 )
    print("****************************")

In [22]:
df_training_data, df_training_labels, df_topredict_data = load_train_and_test_data()

In [23]:
print("*****************************************\nBrief Stats of each column\n*****************************************")
print(df_training_data.describe())

#getting number of nonzeros in each column
print("\n*****************************************\nnumber of nonzeros in each column\n*****************************************")
print(df_training_data.astype(bool).sum(axis=0))

# getting no. of nulls in each column
print("\n*****************************************\nno. of nulls in each column\n*****************************************")
print(df_training_data.isnull().sum())

*****************************************
Brief Stats of each column
*****************************************
                 id     amount_tsh    gps_height     longitude      latitude  \
count  59400.000000   59400.000000  59400.000000  59400.000000  5.940000e+04   
mean   37115.131768     317.650385    668.297239     34.077427 -5.706033e+00   
std    21453.128371    2997.574558    693.116350      6.567432  2.946019e+00   
min        0.000000       0.000000    -90.000000      0.000000 -1.164944e+01   
25%    18519.750000       0.000000      0.000000     33.090347 -8.540621e+00   
50%    37061.500000       0.000000    369.000000     34.908743 -5.021597e+00   
75%    55656.500000      20.000000   1319.250000     37.178387 -3.326156e+00   
max    74247.000000  350000.000000   2770.000000     40.345193 -2.000000e-08   

        num_private   region_code  district_code    population  \
count  59400.000000  59400.000000   59400.000000  59400.000000   
mean       0.474141     15.297003   

In [24]:
#analyze_unique_values_for_column(df_training_data, "funder")
#analyze_unique_values_for_column(df_training_data, "installer")
#analyze_unique_values_for_column(df_training_data, "wpt_name")
#analyze_unique_values_for_column(df_training_data, "basin")
#analyze_unique_values_for_column(df_training_data, "subvillage")
#analyze_unique_values_for_column(df_training_data, "region")
#analyze_unique_values_for_column(df_training_data, "region_code")
#analyze_unique_values_for_column(df_training_data, "district_code")
#analyze_unique_values_for_column(df_training_data, "lga")
#analyze_unique_values_for_column(df_training_data, "ward")
#analyze_unique_values_for_column(df_training_data, "recorded_by")
#analyze_unique_values_for_column(df_training_data, "scheme_management")
#analyze_unique_values_for_column(df_training_data, "scheme_name")
#analyze_unique_values_for_column(df_training_data, "extraction_type")
#analyze_unique_values_for_column(df_training_data, "extraction_type_group")
#analyze_unique_values_for_column(df_training_data, "extraction_type_class")
#analyze_unique_values_for_column(df_training_data, "management")
#analyze_unique_values_for_column(df_training_data, "management_group")
#analyze_unique_values_for_column(df_training_data, "management_group")
#analyze_unique_values_for_column(df_training_data, "payment")
#analyze_unique_values_for_column(df_training_data, "payment_type")
#analyze_unique_values_for_column(df_training_data, "management_group")
#analyze_unique_values_for_column(df_training_data, "water_quality")
#analyze_unique_values_for_column(df_training_data, "quality_group")
#analyze_unique_values_for_column(df_training_data, "quantity")
#analyze_unique_values_for_column(df_training_data, "quantity_group")
#analyze_unique_values_for_column(df_training_data, "source")
#analyze_unique_values_for_column(df_training_data, "source_type")
#analyze_unique_values_for_column(df_training_data, "source_class")
#analyze_unique_values_for_column(df_training_data, "source_class")
#analyze_unique_values_for_column(df_training_data, "waterpoint_type")
#analyze_unique_values_for_column(df_training_data, "waterpoint_type_group")

In [25]:
#analyze_in_detail_unique_values_for_column(df_training_data, "population")

In [26]:
#analyze_in_detail_unique_values_for_column(df_training_data, "management")

In [27]:
#analyze_in_detail_unique_values_for_column(df_training_data, "management_group")

In [28]:
#analyze_in_detail_unique_values_for_column(df_training_data, "extraction_type_class")

In [29]:
del df_training_labels['id']

#df_training_data, df_topredict_data = dates(df_training_data, df_topredict_data)
df_training_data = transform_date_recorded_to_month_and_year(df_training_data)
df_topredict_data = transform_date_recorded_to_month_and_year(df_topredict_data)

df_training_data, df_topredict_data = ohe_month_and_year_recorded(df_training_data, df_topredict_data)

In [30]:
# impute missing construction year with median construction year
df_training_data = impute_missing_construction_year(df_training_data)
df_topredict_data = impute_missing_construction_year(df_topredict_data)

C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
# the fields public_meeting and permit are boolean, but there are many missing values 
# (3334 in public_meeting and 3056 in permit)
# impute these missing values with FALSE
df_training_data = impute_missing_booleans(df_training_data, "public_meeting")
df_topredict_data = impute_missing_booleans(df_topredict_data, "public_meeting")

df_training_data = impute_missing_booleans(df_training_data, "permit")
df_topredict_data = impute_missing_booleans(df_topredict_data, "permit")

In [32]:
"""
fill in the nulls for ['longitude', 'latitude', 'gps_height', 'population'] by using means from 
['subvillage', 'district_code', 'basin'], and lastly the overall mean
"""
df_training_data, df_topredict_data = locs(df_training_data, df_topredict_data)

In [33]:
df_training_data['population'] = np.log(df_training_data['population'])
df_topredict_data['population'] = np.log(df_topredict_data['population'])

In [34]:
# drop unwanted columns 
columns_to_drop = ['id','amount_tsh',  'num_private', 'region', 'quantity', 'quality_group', 'source_type', 'payment', 
'waterpoint_type_group', 'extraction_type_group', 'recorded_by']
df_training_data = drop_columns(df_training_data, columns_to_drop)
df_topredict_data = drop_columns(df_topredict_data, columns_to_drop)

In [35]:
df_training_data, df_topredict_data = small_n2(df_training_data, df_topredict_data)

In [36]:
df_training_data, df_topredict_data = lda(df_training_data, df_topredict_data, df_training_labels, cols = ['gps_height', 'latitude', 'longitude'])

In [37]:
# One hot encode the columns that have string values
df_training_data,df_topredict_data = one_hot_encode(df_training_data, df_topredict_data)
#df_topredict_data = one_hot_encode1(df_topredict_data)

In [38]:
print(len(df_training_data.columns))

537


In [39]:
rf = RandomForestClassifier(criterion='gini', min_samples_split=6, n_estimators=1000, max_features='auto',
     oob_score=True, random_state=1, n_jobs=-1)
rf.fit(df_training_data, df_training_labels.values.ravel())
print ("%.4f" % rf.oob_score_)

0.8122


In [40]:
predictions = rf.predict(df_topredict_data)

In [41]:
df_predictions = pd.DataFrame(predictions)
df_predictions

,0
0,non functional
1,functional
2,functional
3,non functional
4,functional
5,functional
6,functional
7,non functional
8,non functional
9,functional


In [42]:
columns = ['id','status_group']
df_submission = pd.DataFrame(columns=columns)
to_predict_features=pd.read_csv('TestSetValues.csv',parse_dates=True)
to_predict_features = to_predict_features.reset_index(drop=True)
df_predictions = df_predictions.reset_index(drop=True)
df_submission = df_submission.reset_index(drop=True)
df_submission['id'] = to_predict_features['id']
df_submission['status_group'] = df_predictions[0]

In [43]:
df_submission.head

<bound method NDFrame.head of           id             status_group
0      50785           non functional
1      51630               functional
2      17168               functional
3      45559           non functional
4      49871               functional
5      52449               functional
6      24806               functional
7      28965           non functional
8      36301           non functional
9      54122               functional
10       419               functional
11     45750           non functional
12       653           non functional
13     14017           non functional
14     44607               functional
15     40228               functional
16     27714               functional
17     28785           non functional
18     28330               functional
19     18532           non functional
20     69961               functional
21     55083           non functional
22      8691           non functional
23     30331           non functional
24     70970        

In [44]:
df_submission.shape

(14850, 2)

In [45]:
df_submission.to_csv("submission_ab_kn.csv", sep=",", index = False)

In [46]:
# Convert string labels to numerics
#label_map = {"functional": 1, "functional needs repair": 2, "non functional": 3}
#df_training_data['status_group_num']= df_training_data['status_group'].map(label_map).astype(int)

# do sanity check
#df_training_data[['id', 'amount_tsh', 'status_group', 'status_group_num']].head(25)

In [47]:
# save the training labels into a df
#df_training_labels_str = df_training_data['status_group']
#df_training_labels_num = np.array(df_training_data['status_group_num'])

In [48]:
# drop the training labels from the training set
#df_training_data= df_training_data.drop('status_group', axis = 1)
#df_training_data= df_training_data.drop('status_group_num', axis = 1)

In [49]:
#def one_hot_encode(X_train, X_test):
#    columns = [i for i in X_train.columns if type(X_train[i].iloc[0]) == str]
#    for column in columns:
#        X_train[column].fillna('NULL', inplace = True)
#        good_cols = [column+'_'+i for i in X_train[column].unique() if i in X_test[column].unique()]
#        X_train = pd.concat((X_train, pd.get_dummies(X_train[column], prefix = column)[good_cols]), axis = 1)
#        X_test = pd.concat((X_test, pd.get_dummies(X_test[column], prefix = column)[good_cols]), axis = 1)
#        del X_train[column]
#        del X_test[column]
#    return X_train, X_test
#

In [50]:
# impute missing population 
#df_training_data = impute_missing_population1(df_training_data)
#df_topredict_data = impute_missing_population1(df_topredict_data)

In [51]:
#analyze_in_detail_unique_values_for_column(df_training_data, 'population_cat')

In [52]:
"""
X_train, X_test, y_train, y_test = 
    train_test_split(df_training_data, df_training_labels_num, test_size = 0.25, random_state = 42)
"""

'\nX_train, X_test, y_train, y_test = \n    train_test_split(df_training_data, df_training_labels_num, test_size = 0.25, random_state = 42)\n'

In [53]:
"""
features_to_consider =
    ['amount_tsh', 'gps_height', 'longitude', 'latitude', 'region_code', 'district_code', 'population', 'construction_year']
"""

"\nfeatures_to_consider =\n    ['amount_tsh', 'gps_height', 'longitude', 'latitude', 'region_code', 'district_code', 'population', 'construction_year']\n"

In [54]:
"""
#def run_random_forest_predictor(X_train1, y_train1, X_test1, y_test1, features_to_consider):
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
df1 = X_train[features_to_consider]
rf.fit(df1, y_train)
predictions = rf.predict(X_test[features_to_consider])
# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
"""

"\n#def run_random_forest_predictor(X_train1, y_train1, X_test1, y_test1, features_to_consider):\nrf = RandomForestRegressor(n_estimators = 1000, random_state = 42)\n# Train the model on training data\ndf1 = X_train[features_to_consider]\nrf.fit(df1, y_train)\npredictions = rf.predict(X_test[features_to_consider])\n# Calculate the absolute errors\nerrors = abs(predictions - y_test)\n# Print out the mean absolute error (mae)\nprint('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')\n"